<a href="https://colab.research.google.com/github/linkedin/detext/blob/cls-demo/text_classification_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification Demo





# Introduction

In this demo, we will show how to build a production-ready CNN model for text classification using the DeText framework. The task is for query intent classification.

## Tutorial setup


### Outline
* Environment setup
* Data preprocessing
* Model training
* Inference with trained model


###   Dataset
We will use the publicly available Natural Language Understanding benchmark dataset ([nlu-benchmark](https://github.com/snipsco/nlu-benchmark/tree/master/2017-06-custom-intent-engines)). The labeled data contains 7 intents including the following:
* SearchCreativeWork (e.g. Find me the I, Robot television show),
* GetWeather (e.g. Is it windy in Boston, MA right now?),
* BookRestaurant (e.g. I want to book a highly rated restaurant for me and my boyfriend tomorrow night),
* ~~PlayMusic (e.g. Play the last track from Beyoncé off Spotify),~~ This is not included in this tutorial due to dataset download issues.
* AddToPlaylist (e.g. Add Diamonds to my roadtrip playlist)
* RateBook (e.g. Give 6 stars to Of Mice and Men)
* SearchScreeningEvent (e.g. Check the showtimes for Wonder Woman in Paris)

We will use the 6 valid classes to train the query intent classifier.

### Model
Users will see how to train a CNN model for text classification with DeText. The trained model is ready for online inference in production search systems.


### Time required
appox. 20min



# Set up the environment
We first need to install detext using pip.

In [ ]:
!pip install detext==2.0.8


# Download and preprocess the dataset
In this section, we download the dataset and perform some basic preprocessing to the text data, including lowercasing, whitespace normalization, etc.



In [ ]:
!git clone https://github.com/snipsco/nlu-benchmark.git

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

In [ ]:
# The constants
CLASSES = [
           "AddToPlaylist",
           "BookRestaurant",
           "GetWeather",
           "RateBook",
           "SearchCreativeWork",
           "SearchScreeningEvent"
           ]
# See https://github.com/linkedin/detext/blob/master/user_guide/TRAINING.md for more details on the feature naming format required by DeText
QUERY_FIELDNAME = "doc_query"
WIDE_FTR_FIELDNAME = "wide_ftrs"
LABEL_FIELDNAME = "label"
VOCAB_FILE = "vocab.txt"

In [ ]:
import json


train_raw = {}
test_raw = {}

for c in CLASSES:
  with open('/content/nlu-benchmark/2017-06-custom-intent-engines/{}/train_{}_full.json'.format(c, c)) as f:
    train_raw[c] = json.load(f)[c]

  with open('/content/nlu-benchmark/2017-06-custom-intent-engines/{}/validate_{}.json'.format(c, c)) as f:
    test_raw[c] = json.load(f)[c]

  print("Training samples for {}: {}".format(c, len(train_raw[c])), ", test samples for {}: {}".format(c, len(test_raw[c])))


In [ ]:
import re
import string

def proprocess_data(data_raw):
  X = []
  y = []
  for c in CLASSES:
    for sample in data_raw[c]:
      tokens = []
      for d in sample['data']:
        t = d['text'].strip().lower()
        t = re.sub(r'([%s])' % re.escape(string.punctuation), r' \1 ', t)
        t = re.sub(r'\\.', r' ', t)
        t = re.sub(r'\s+', r' ', t)
        tokens.append(t)
      sentence = ' '.join(tokens)

      X.append(sentence)
      y.append(CLASSES.index(c))
  return X, y


X_train, y_train = proprocess_data(train_raw)
X_test, y_test = proprocess_data(test_raw)

print(X_train[:5])
print(y_train[:5])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


In [ ]:
# Stats
print('Total train samples: {}'.format(len(y_train)))
print('Total dev samples: {}'.format(len(y_dev)))
print('Total test samples: {}'.format(len(y_test)))

# Prepare DeText dataset

Now that we have preprocessed the datasets and prepared the train/ dev/ test splits, let's convert the data samples into the correct format that DeText accepts.

The text inputs are converted to `byte_list` features and labels are converted to `float_list` features. The datasets are store in `tfrecord` format. For more information on the input format of DeText, please checkout [the training manual](https://github.com/linkedin/detext/blob/master/user_guide/TRAINING.md).

In [ ]:
import tensorflow as tf

def create_float_feature(value):
    feature = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    return feature


def create_byte_feature(value):
    feature = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    return feature


def create_tfrecords(X, y, output_file):
  assert len(X) == len(y)
  total_records = 0
  writer = tf.python_io.TFRecordWriter(output_file)
  for q, c in zip(X, y):
    features = {}
    features[QUERY_FIELDNAME] = create_byte_feature(str.encode(q))
    features[LABEL_FIELDNAME] = create_float_feature(c)
    features[WIDE_FTR_FIELDNAME] = create_float_feature(len(q.split()))
    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    if total_records < 2:
      print(tf_example)
    writer.write(tf_example.SerializeToString())
    total_records += 1
  print("processed {} records".format(total_records))

create_tfrecords(X_train, y_train, 'train.tfrecord')
create_tfrecords(X_dev, y_dev, 'dev.tfrecord')
create_tfrecords(X_test, y_test, 'test.tfrecord')

In [ ]:
# Generate vocab
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_train)
vocab = ['[PAD]', '[UNK]']
vocab.extend(list(vectorizer.vocabulary_.keys()))

# Write vocab to file
with open(VOCAB_FILE, 'w') as f:
  for v in vocab:
    f.write("{}\n".format(v))


# DeText training
We will train a multi-class classification model using the DeText framework.



In [ ]:
# See https://github.com/linkedin/detext/blob/master/user_guide/TRAINING.md for more details on the parameters

from detext.run_detext import run_detext,DetextArg

args = DetextArg(num_classes=6,
                ftr_ext="cnn",
                num_filters=50,
                num_units=64,
                num_wide=1,
                optimizer="bert_adam", # same AdamWeightDecay optimizer as in BERT training
                learning_rate=0.001,
                max_len=16,
                min_len=3,
                use_deep=True,
                num_train_steps=300,
                steps_per_stats=1,
                steps_per_eval=30,
                train_batch_size=64,
                test_batch_size=64,
                pmetric="accuracy",
                all_metrics=["accuracy", "confusion_matrix"],
                vocab_file=VOCAB_FILE,
                feature_names=["label","doc_query","wide_ftrs"],
                train_file="train.tfrecord",
                dev_file="dev.tfrecord",
                test_file="test.tfrecord",
                out_dir="output")
run_detext(args)


# Let's check the predictions

In [ ]:
import glob
from tensorflow.contrib import predictor
saved_model_path = glob.glob('output/export/best_accuracy/*')[0]

print(saved_model_path)

In [ ]:
!saved_model_cli show --dir output/export/best_accuracy/* --tag_set serve --signature_def serving_default

In [ ]:
import numpy as np
predict_fn = predictor.from_saved_model(saved_model_path)

def predict_class(query):
  predictions = predict_fn(
                  {"doc_query": [query],
                   "wide_ftrs": [[len(query.split())]],
                   # dummy pass-through input for label field -- this is not used in making predictions
                   "label": [1.0]})
  probabilities = predictions['multiclass_probabilities'][0]
  predicted_class = CLASSES[np.argmax(probabilities)]
  print("query \"{}\", predicted intent \"{}\".".format(query, predicted_class))

In [ ]:
# GetWeather intent
predict_class("how's the weather in Mountain View")

# BookRestaurant intent
predict_class("I need a table for 4")

# PlayMusic intent
predict_class("put Jean Philippe Goncalves onto my running to rock 170 to 190 bpm")

# SearchScreeningEvent intent
predict_class("Find the schedule for The Comedian")

# References

\[1\] Dataset: Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces, [github repo](https://github.com/snipsco/nlu-benchmark)

